## Étape 1 : Import des bibliothèques et chargement des fichiers source

Cette étape initialise l'environnement Python, charge les fichiers Excel nécessaires (tab_source et SPIRE recap) et initialise le convertisseur de devises.


In [44]:
# Chargement des fichiers Excel depuis les dossiers spécifiques (peu importe leur nom et format)
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime, date
from pathlib import Path
from currency_converter import CurrencyConverter

# Extensions supportées pour les fichiers Excel et CSV
EXCEL_EXTENSIONS = ['.xlsx', '.xlsm', '.xls', '.xlsb', '.xltx', '.xltm', '.xlt', '.csv']

# Fonction pour trouver le fichier Excel/CSV dans un dossier
def find_excel_file(directory):
    """Trouve le premier fichier Excel ou CSV dans le dossier spécifié"""
    if not os.path.exists(directory):
        raise FileNotFoundError(f"Dossier '{directory}' introuvable")
    
    # Chercher tous les fichiers avec les extensions supportées
    excel_files = []
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if os.path.isfile(file_path):
            file_ext = os.path.splitext(file)[1].lower()
            if file_ext in EXCEL_EXTENSIONS:
                excel_files.append(file)
    
    if len(excel_files) == 0:
        raise FileNotFoundError(f"Aucun fichier Excel/CSV trouvé dans '{directory}'")
    if len(excel_files) > 1:
        raise ValueError(f"Plusieurs fichiers Excel/CSV trouvés dans '{directory}': {excel_files}")
    
    return os.path.join(directory, excel_files[0])

# Fonction pour charger un fichier Excel/CSV selon son extension
def load_excel_file(file_path, header=0, sheet_name=None):
    """Charge un fichier Excel ou CSV selon son extension
    
    Parameters:
    -----------
    file_path : str
        Chemin vers le fichier à charger
    header : int, default=0
        Numéro de ligne à utiliser comme en-têtes (0 = première ligne)
    sheet_name : str ou int, optional
        Nom ou index de la feuille à charger (uniquement pour Excel, pas CSV)
        Si None, ne passe pas le paramètre pour charger la première feuille par défaut
    """
    file_ext = os.path.splitext(file_path)[1].lower()
    
    if file_ext == '.csv':
        return pd.read_csv(file_path, header=header)
    else:
        # Ne pas passer sheet_name si c'est None pour éviter les problèmes avec les fichiers à une seule feuille
        if sheet_name is None:
            return pd.read_excel(file_path, header=header)
        else:
            return pd.read_excel(file_path, header=header, sheet_name=sheet_name)

# Initialisation du convertisseur de devises
converter = CurrencyConverter()

# Chargement des fichiers depuis les dossiers spécifiques
tab_source_path = find_excel_file('data/Extract LLM de FT')
spire_recap_path = find_excel_file('data/Spire Recap')


ini = load_excel_file(tab_source_path)
df_spire = load_excel_file(spire_recap_path, header=1, sheet_name="SPIRE recap")  # En-têtes à la ligne 2 (index 1)



tableau_final = pd.DataFrame()

# Confirmation de l'étape 1
print("=" * 80)
print("ÉTAPE 1 : Import et chargement des fichiers - TERMINÉE")
print("=" * 80)
print(f"\n✓ Fichier source chargé: {tab_source_path}")
print(f"✓ Fichier SPIRE recap chargé: {spire_recap_path}")
print(f"\nDataFrame 'ini' (source):")
print(f"  - Nombre de lignes: {len(ini)}")
print(f"  - Nombre de colonnes: {len(ini.columns)}")
print(f"  - Colonnes: {list(ini.columns)}")
print(f"\nDataFrame 'df_spire':")
print(f"  - Nombre de lignes: {len(df_spire)}")
print(f"  - Nombre de colonnes: {len(df_spire.columns)}")
print(f"\nAperçu du DataFrame 'ini':")
print(ini.head())
print(f"\n✓ Tableau final initialisé (vide)")
print("=" * 80)


ℹ API non disponible - utilisation uniquement du Excel de secours
ℹ Aucun fichier Excel de secours trouvé
ÉTAPE 1 : Import et chargement des fichiers - TERMINÉE

✓ Fichier source chargé: data/Extract LLM de FT/tab source.xlsx
✓ Fichier SPIRE recap chargé: data/Spire Recap/SPIRE Recap - V20.xlsx

DataFrame 'ini' (source):
  - Nombre de lignes: 4
  - Nombre de colonnes: 32
  - Colonnes: ['Filename', 'ISIN', 'Compartment', 'Dealer', 'Listing', 'Issue Price', 'Issue Date', 'Maturity Date', 'Nominal', 'Currency', 'Collat Type', 'Collat Name', 'Collat Coupon', 'Collat CCY', 'Collat ISIN', 'Collat Amount', 'Inflation Linked? (Collat)', 'Type of coupon', 'Fixed Note', 'Floating Note', 'Variable-linked Note', 'Interest Basis', 'Final Redemption', 'Payoff Type', 'Payoff CCY', 'Issuer Call Redemption Date', 'Issuer Call Redemption Amount', 'Noteholder Representative', 'CDS Linked?', 'Green Bond Linked?', 'Social Bond Linked?', 'Notice Type']

DataFrame 'df_spire':
  - Nombre de lignes: 4
  - Nomb

## Étape 2 : Définition des dictionnaires de correspondance

Définition des mappings pour normaliser les noms de dealers et de collatéraux.


In [45]:
# Définition des dictionnaires de correspondance (dealers, collatéraux)
dealer_name = {
    "BNP Paribas": "BNPP",
}

collat_name = {
    "Republic of Italy": "BTP",
}

# Confirmation de l'étape 2
print("=" * 80)
print("ÉTAPE 2 : Définition des dictionnaires de correspondance - TERMINÉE")
print("=" * 80)
print(f"\n✓ Dictionnaire 'dealer_name': {dealer_name}")
print(f"✓ Dictionnaire 'collat_name': {collat_name}")
print("=" * 80)


ÉTAPE 2 : Définition des dictionnaires de correspondance - TERMINÉE

✓ Dictionnaire 'dealer_name': {'BNP Paribas': 'BNPP'}
✓ Dictionnaire 'collat_name': {'Republic of Italy': 'BTP'}


## Étape 3 : Définition des fonctions utilitaires

Création des fonctions helper pour les conversions de types (string, float, int, date) et le formatage des données (dates, pourcentages, formules de taux, etc.).


In [ ]:
# Fonctions utilitaires pour conversions de types robustes
def safe_str(value):
    """Convertit une valeur en string de manière sûre"""
    if pd.isna(value):
        return ""
    return str(value).strip()

def safe_float(value, default=np.nan):
    """Convertit une valeur en float de manière sûre"""
    if pd.isna(value):
        return default
    try:
        return float(value)
    except (ValueError, TypeError):
        return default

def safe_int(value, default=None):
    """Convertit une valeur en int de manière sûre"""
    if pd.isna(value):
        return default
    try:
        return int(float(value))
    except (ValueError, TypeError):
        return default

def safe_date(value, default=None):
    """Convertit une valeur en date de manière sûre"""
    if pd.isna(value):
        return default
    if isinstance(value, (date, datetime)):
        return value.date() if isinstance(value, datetime) else value
    try:
        if isinstance(value, str):
            return pd.to_datetime(value).date()
        return pd.to_datetime(value).date()
    except (ValueError, TypeError):
        return default

def safe_compare(value1, value2):
    """Compare deux valeurs en les convertissant en string si nécessaire"""
    if pd.isna(value1) or pd.isna(value2):
        return False
    try:
        # Essayer comparaison directe
        return value1 == value2
    except:
        # Si échec, comparer en string
        return safe_str(value1) == safe_str(value2)

# Définition des fonctions utilitaires (formatage dates, parsing formules de taux, etc.)
def get_creation_date_from_spire(isin):
    """Récupère la Creation Date depuis df_spire en utilisant l'ISIN"""
    if pd.isna(isin):
        return None
    
    isin_str = safe_str(isin)
    if not isin_str:
        return None
    
    # Chercher la colonne ISIN dans df_spire
    isin_col = None
    for col in df_spire.columns:
        col_str = safe_str(col).upper()
        if 'ISIN' in col_str:
            isin_col = col
            break
    
    if isin_col is None:
        return None
    
    # Chercher la ligne correspondante
    matching_rows = df_spire[df_spire[isin_col].apply(lambda x: safe_compare(x, isin_str))]
    
    if matching_rows.empty:
        return None
    
    # Chercher la colonne Creation Date
    creation_date_col = None
    for col in df_spire.columns:
        col_str = safe_str(col).upper()
        if 'CREATION' in col_str and 'DATE' in col_str:
            creation_date_col = col
            break
    
    if creation_date_col is None:
        return None
    
    creation_date = matching_rows.iloc[0][creation_date_col]
    return safe_date(creation_date)

def format_date_dd_mon_yy(date_value):
    """Formate une date au format DD-Mon-YY, gère tous les types"""
    if pd.isna(date_value):
        return ""
    try:
        date_obj = safe_date(date_value)
        if date_obj is None:
            return ""
        if isinstance(date_obj, date):
            return date_obj.strftime("%d-%b-%y")
        return pd.to_datetime(date_obj).strftime("%d-%b-%y")
    except:
        return ""

def format_percentage(value):
    """Formate une valeur en pourcentage, gère tous les types"""
    if pd.isna(value):
        return ""
    try:
        percentage = safe_float(value) * 100
        if pd.isna(percentage):
            return ""
        return f"{percentage:.2f}%"
    except:
        return ""

def parse_rate_formula(formula_str):
    """Parse une formule de taux, gère tous les types"""
    if pd.isna(formula_str):
        return None
    formula = safe_str(formula_str)
    result = {'rate_base': '', 'spread': '', 'floor': '', 'cap': ''}
    
    if 'Floor at' in formula or 'Cap at' in formula:
        rate_match = re.search(r'([A-Z0-9\s\-]+?)\s*\+\s*([\d.]+)%', formula)
        if rate_match:
            result['rate_base'] = rate_match.group(1).strip()
            result['spread'] = rate_match.group(2)
        floor_match = re.search(r'Floor\s+at\s+([\d.]+)%', formula, re.IGNORECASE)
        if floor_match:
            result['floor'] = floor_match.group(1)
        cap_match = re.search(r'Cap\s+at\s+([\d.]+)%', formula, re.IGNORECASE)
        if cap_match:
            result['cap'] = cap_match.group(1)
    elif 'MIN' in formula.upper() and 'MAX' in formula.upper():
        min_match = re.search(r'MIN\s*\[?\s*([\d.]+)%', formula, re.IGNORECASE)
        if min_match:
            result['cap'] = min_match.group(1)
        max_match = re.search(r'MAX\s*\[?\s*([\d.]+)%\s*[;,]?\s*([^)]+)\+?\s*([\d.]+)%?', formula, re.IGNORECASE)
        if max_match:
            result['floor'] = max_match.group(1)
            rate_part = max_match.group(2).strip() if len(max_match.groups()) >= 2 else ''
            spread_part = max_match.group(3) if len(max_match.groups()) >= 3 else ''
            rate_base_match = re.search(r'([A-Z0-9\s\-]+?)(?:\s*\+\s*[\d.]+%?)?$', rate_part)
            if rate_base_match:
                result['rate_base'] = rate_base_match.group(1).strip()
            if spread_part:
                result['spread'] = spread_part
    
    if not result['rate_base']:
        rate_base_match = re.search(r'([A-Z]{2,}[0-9A-Z\s\-]*?)(?:\s*\+\s*[\d.]+%?)?', formula)
        if rate_base_match:
            result['rate_base'] = rate_base_match.group(1).strip()
    
    if 'EURIBOR' in result['rate_base'].upper():
        result['rate_base'] = 'EUR6M'
    
    return result

def format_floating_coupon(floating_note):
    if pd.isna(floating_note):
        return ""
    parsed = parse_rate_formula(floating_note)
    if not parsed:
        return ""
    rate_base = parsed['rate_base'] or 'Rate'
    spread = parsed['spread'] or '0'
    floor = parsed['floor'] or '0'
    cap = parsed['cap'] or ''
    
    # Remplacer "0.00%" par "0" (exception à la règle des deux décimales)
    def clean_zero_percent(s):
        if s == '0.00%' or s == '0.00':
            return '0'
        return s
    
    spread = clean_zero_percent(spread)
    floor = clean_zero_percent(floor)
    cap = clean_zero_percent(cap)
    
    if cap:
        return f"Y1 - End: Min({cap}% ; Max({rate_base} + {spread}% ; {floor}%))"
    else:
        return f"Y1 - End: Max({rate_base} + {spread}% ; {floor}%)"

def format_variable_linked_coupon(fixed_note, variable_note):
    """Formate un coupon variable-linked, gère tous les types"""
    if pd.isna(fixed_note) or pd.isna(variable_note):
        return ""
    parsed = parse_rate_formula(variable_note)
    if not parsed:
        return ""
    rate_base = parsed['rate_base'] or 'Rate'
    spread = parsed['spread'] or '0'
    floor = parsed['floor'] or '0'
    cap = parsed['cap'] or ''
    fixed_str = safe_str(fixed_note)
    
    # Remplacer "0.00%" par "0" (exception à la règle des deux décimales)
    def clean_zero_percent(s):
        if s == '0.00%' or s == '0.00':
            return '0'
        return s
    
    spread = clean_zero_percent(spread)
    floor = clean_zero_percent(floor)
    cap = clean_zero_percent(cap)
    
    if cap:
        return f"Y1 - Y2: {fixed_str}% p.a.\nY2 - End: {fixed_str}% p.a. or Min({cap}% ; Max({rate_base} + {spread}% ; {floor}%)) p.a."
    else:
        return f"Y1 - Y2: {fixed_str}% p.a.\nY2 - End: {fixed_str}% p.a. or Max({rate_base} + {spread}% ; {floor}%) p.a."

def process_pipe_separated(value, formatter=None):
    """Traite les valeurs séparées par |, gère tous les types"""
    if pd.isna(value):
        return ""
    value_str = safe_str(value)
    if '|' not in value_str:
        if formatter:
            return formatter(value_str)
        return value_str
    parts = [part.strip() for part in value_str.split('|')]
    if formatter:
        formatted_parts = [formatter(part) for part in parts]
        return " | ".join(formatted_parts)
    else:
        return " | ".join(parts)

def format_number_with_spaces(value):
    """Formate un nombre avec séparateur de milliers (espaces), gère tous les types"""
    if pd.isna(value):
        return ""
    try:
        num = safe_int(value)
        if num is None:
            return ""
        return f"{num:,}".replace(",", " ")
    except:
        return ""

# Confirmation de l'étape 3
print("=" * 80)
print("ÉTAPE 3 : Définition des fonctions utilitaires - TERMINÉE")
print("=" * 80)
print("\n✓ Fonctions de conversion définies: safe_str, safe_float, safe_int, safe_date, safe_compare")
print("✓ Fonctions de formatage définies: format_date_dd_mon_yy, format_percentage, format_number_with_spaces")
print("✓ Fonctions de parsing définies: parse_rate_formula, format_floating_coupon, format_variable_linked_coupon")
print("✓ Fonctions utilitaires définies: get_creation_date_from_spire, process_pipe_separated")
print("=" * 80)


ÉTAPE 3 : Définition des fonctions utilitaires - TERMINÉE

✓ Fonctions de conversion définies: safe_str, safe_float, safe_int, safe_date, safe_compare
✓ Fonctions de formatage définies: format_date_dd_mon_yy, format_percentage, format_number_with_spaces
✓ Fonctions de parsing définies: parse_rate_formula, format_floating_coupon, format_variable_linked_coupon
✓ Fonctions utilitaires définies: get_creation_date_from_spire, process_pipe_separated


## Étape 4 : Création de la colonne Dealer

Mapping des noms de dealers depuis le fichier source avec remplacement via le dictionnaire de correspondance.


In [47]:
# Création colonne Dealer : mapping depuis ini avec remplacement via dictionnaire
tableau_final['Dealer'] = ini['Dealer'].map(lambda x: dealer_name.get(safe_str(x), safe_str(x)) if pd.notna(x) else "")

# Confirmation de l'étape 4
print("=" * 80)
print("ÉTAPE 4 : Création de la colonne Dealer - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Dealer' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Dealer':")
print(tableau_final[['Dealer']].head(10))
print(f"\nValeurs uniques dans 'Dealer': {tableau_final['Dealer'].unique().tolist()}")
print("=" * 80)


ÉTAPE 4 : Création de la colonne Dealer - TERMINÉE

✓ Colonne 'Dealer' créée avec 4 lignes

Aperçu de la colonne 'Dealer':
                                  Dealer
0                                   BNPP
1                          HSBC Bank plc
2                         J.P. Morgan SE
3  Nomura Financial Products Europe GmbH

Valeurs uniques dans 'Dealer': ['BNPP', 'HSBC Bank plc', 'J.P. Morgan SE', 'Nomura Financial Products Europe GmbH']


## Étape 5 : Création de la colonne ISIN (all)

Copie directe de la colonne ISIN depuis le fichier source.


In [48]:
# Création colonne ISIN (all) : copie directe depuis ini
tableau_final['ISIN (all)'] = ini['ISIN']

# Confirmation de l'étape 5
print("=" * 80)
print("ÉTAPE 5 : Création de la colonne ISIN (all) - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'ISIN (all)' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'ISIN (all)':")
print(tableau_final[['ISIN (all)']].head(10))
print(f"\nNombre de valeurs uniques: {tableau_final['ISIN (all)'].nunique()}")
print("=" * 80)


ÉTAPE 5 : Création de la colonne ISIN (all) - TERMINÉE

✓ Colonne 'ISIN (all)' créée avec 4 lignes

Aperçu de la colonne 'ISIN (all)':
     ISIN (all)
0  XS2030639145
1  XS2135238659
2  XS2041123880
3  XS3205809513

Nombre de valeurs uniques: 4


## Étape 6 : Création des colonnes N° Issuance et Creation Date

Enrichissement du tableau final avec les données du fichier SPIRE Recap via un mapping basé sur l'ISIN.


In [49]:
# Création des colonnes N° Issuance et Creation Date via mapping SPIRE Recap
# Initialiser les colonnes si elles n'existent pas encore
if 'N° Issuance' not in tableau_final.columns:
    tableau_final['N° Issuance'] = ""
if 'Creation Date' not in tableau_final.columns:
    tableau_final['Creation Date'] = ""

# Importer le fichier SPIRE Recap pour compléter "N° Issuance" et "Creation Date"
try:

    df_spire_recap = df_spire
    
    print(f"\n📐 Shape du fichier SPIRE Recap : {df_spire_recap.shape}")
    
    # ------------------------------------------------------------------
    # Fonction pour convertir une date au format "15-May-56"
    # ------------------------------------------------------------------
    def convert_date_format_spire(value):
        if value == "" or pd.isna(value) or str(value).lower() == "nan":
            return ""
        try:
            # Déjà un timestamp pandas
            if isinstance(value, pd.Timestamp):
                date_obj = value.to_pydatetime()
            # Déjà un datetime Python
            elif isinstance(value, datetime):
                date_obj = value
            # Sinon, on tente de parser une chaîne
            else:
                value_str = str(value).strip()
                date_formats = [
                    "%d-%m-%Y", "%d/%m/%Y",
                    "%d-%m-%y", "%d/%m/%y",
                    "%Y-%m-%d", "%Y/%m/%d",
                    "%d-%b-%Y", "%d-%b-%y",
                ]
                date_obj = None
                for fmt in date_formats:
                    try:
                        date_obj = datetime.strptime(value_str, fmt)
                        break
                    except ValueError:
                        continue
                if date_obj is None:
                    # Dernier recours : parse automatique de pandas
                    date_obj = pd.to_datetime(value_str).to_pydatetime()
            
            formatted = date_obj.strftime("%d-%b-%y")
            return formatted
        except (ValueError, TypeError, AttributeError):
            return ""
    
    # ------------------------------------------------------------------
    # Créer un dictionnaire de mapping basé sur ISIN
    # ------------------------------------------------------------------
    if "ISIN" in df_spire_recap.columns:
        isin_to_data = {}
        for idx, row in df_spire_recap.iterrows():
            isin = str(row["ISIN"]).strip() if pd.notna(row["ISIN"]) else ""
            if not isin or isin.lower() == "nan":
                continue
            
            n_issuance = ""
            creation_date = ""
            
            # Chercher la colonne pour "N° Issuance"
            if "N° Issuance" in df_spire_recap.columns:
                n_issuance_val = row["N° Issuance"]
                if pd.notna(n_issuance_val):
                    n_issuance = str(n_issuance_val).strip()
            else:
                # Fallback : essayer de deviner la colonne
                for col in df_spire_recap.columns:
                    col_str = str(col).strip()
                    if (
                        "issuance" in col_str.lower()
                        or "n°" in col_str.lower()
                        or "numero" in col_str.lower()
                    ):
                        n_issuance_val = row[col]
                        if pd.notna(n_issuance_val):
                            n_issuance = str(n_issuance_val).strip()
                        break
            
            # Chercher la colonne pour "Creation Date"
            if "Creation Date" in df_spire_recap.columns:
                creation_date_val = row["Creation Date"]
                if pd.notna(creation_date_val):
                    creation_date = convert_date_format_spire(creation_date_val)
            else:
                # Fallback : essayer de deviner la colonne
                for col in df_spire_recap.columns:
                    col_str = str(col).strip().lower()
                    if ("creation" in col_str and "date" in col_str):
                        creation_date_val = row[col]
                        if pd.notna(creation_date_val):
                            creation_date = convert_date_format_spire(creation_date_val)
                        break
            
            isin_to_data[isin] = {
                "N° Issuance": n_issuance,
                "Creation Date": creation_date,
            }
        
        # ------------------------------------------------------------------
        # Remplir les colonnes dans le DataFrame principal
        # ------------------------------------------------------------------
        if "ISIN (all)" in tableau_final.columns:
            count_updated = 0
            for idx, row in tableau_final.iterrows():
                isin_all = (
                    str(row["ISIN (all)"]).strip()
                    if pd.notna(row["ISIN (all)"])
                    else ""
                )
                if (
                    isin_all
                    and isin_all.lower() != "nan"
                    and isin_all in isin_to_data
                ):
                    data = isin_to_data[isin_all]
                    if data["N° Issuance"]:
                        tableau_final.at[idx, "N° Issuance"] = data["N° Issuance"]
                        count_updated += 1
                    if data["Creation Date"]:
                        tableau_final.at[idx, "Creation Date"] = data["Creation Date"]
            
            print(f"✅ {count_updated} lignes enrichies avec les données SPIRE Recap")
        else:
            print("⚠️ Colonne 'ISIN (all)' non trouvée dans le DataFrame principal")
    else:
        print("⚠️ Colonne 'ISIN' non trouvée dans SPIRE Recap")
        
except FileNotFoundError as e:
    print(f"❌ Fichier SPIRE Recap non trouvé : {e}")
except Exception as e:
    print(f"❌ Erreur lors de l'enrichissement : {e}")





📐 Shape du fichier SPIRE Recap : (4, 4)
✅ 4 lignes enrichies avec les données SPIRE Recap


## Étape 7 : Création de la colonne Maturity

Formatage de la date d'échéance depuis le fichier source au format DD-Mon-YY.


In [50]:
# Création colonne Maturity : formatage de la date d'échéance au format DD-Mon-YY
tableau_final['Maturity'] = ini['Maturity Date'].apply(format_date_dd_mon_yy)

# Confirmation de l'étape 7
print("=" * 80)
print("ÉTAPE 7 : Création de la colonne Maturity - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Maturity' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Maturity':")
print(tableau_final[['Maturity']].head(10))
print(f"\nNombre de valeurs non vides: {(tableau_final['Maturity'] != '').sum()}")
print("=" * 80)


ÉTAPE 7 : Création de la colonne Maturity - TERMINÉE

✓ Colonne 'Maturity' créée avec 4 lignes

Aperçu de la colonne 'Maturity':
    Maturity
0  15-May-56
1  25-Jul-53
2  15-Sep-42
3  15-Sep-42

Nombre de valeurs non vides: 4


## Étape 8 : Création de la colonne Currency

Copie directe de la colonne Currency depuis le fichier source.


In [51]:
# Création colonne Currency : copie directe depuis ini
tableau_final['Currency'] = ini['Currency']

# Confirmation de l'étape 8
print("=" * 80)
print("ÉTAPE 8 : Création de la colonne Currency - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Currency' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Currency':")
print(tableau_final[['Currency']].head(10))
print(f"\nValeurs uniques dans 'Currency': {tableau_final['Currency'].unique().tolist()}")
print("=" * 80)


ÉTAPE 8 : Création de la colonne Currency - TERMINÉE

✓ Colonne 'Currency' créée avec 4 lignes

Aperçu de la colonne 'Currency':
  Currency
0      EUR
1      EUR
2      EUR
3      EUR

Valeurs uniques dans 'Currency': ['EUR']


## Étape 9 : Création de la colonne Equiv EUR

Conversion du nominal en EUR en utilisant le convertisseur de devises avec la date de création, puis formatage avec séparateur de milliers.


In [52]:
# Création colonne Equiv EUR : conversion du nominal en EUR avec formatage séparateur de milliers
def convert_to_eur_value(row_idx):
    """Convertit un montant vers EUR en utilisant CurrencyConverter avec Creation Date"""
    row = ini.iloc[row_idx]
    amount = row['Nominal']
    currency = row['Currency']
    isin = row.get('ISIN', '')
    
    if pd.isna(amount) or pd.isna(currency):
        return np.nan
    
    amount_float = safe_float(amount)
    if pd.isna(amount_float):
        return np.nan
    
    currency_str = safe_str(currency).upper()
    if not currency_str:
        return np.nan
    
    if currency_str == 'EUR':
        return amount_float
    
    # Récupérer la Creation Date depuis le tableau final (déjà formatée)
    # Utiliser l'ISIN pour trouver la ligne correspondante dans tableau_final
    target_date = None
    if 'Creation Date' in tableau_final.columns and 'ISIN (all)' in tableau_final.columns:
        # Trouver la ligne dans tableau_final qui correspond à cet ISIN
        matching_rows = tableau_final[tableau_final['ISIN (all)'] == isin]
        if not matching_rows.empty:
            creation_date_str = matching_rows.iloc[0]['Creation Date']
            if pd.notna(creation_date_str) and str(creation_date_str).strip() != '':
                try:
                    # Parser la date au format DD-Mon-YY
                    target_date = datetime.strptime(str(creation_date_str).strip(), "%d-%b-%y").date()
                except Exception as e:
                    # En cas d'erreur de parsing, continuer sans date (utilisera le dernier taux disponible)
                    pass
    
    try:
        converted = converter.convert(amount_float, currency_str, 'EUR', target_date)
        return converted if converted is not None else np.nan
    except:
        return np.nan

tableau_final['Equiv EUR'] = [format_number_with_spaces(convert_to_eur_value(i)) for i in range(len(ini))]

# Confirmation de l'étape 9
print("=" * 80)
print("ÉTAPE 9 : Création de la colonne Equiv EUR - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Equiv EUR' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Equiv EUR':")
print(tableau_final[['Currency', 'Equiv EUR']].head(10))
print(f"\nNombre de valeurs non vides: {(tableau_final['Equiv EUR'] != '').sum()}")
print("=" * 80)


ÉTAPE 9 : Création de la colonne Equiv EUR - TERMINÉE

✓ Colonne 'Equiv EUR' créée avec 4 lignes

Aperçu de la colonne 'Equiv EUR':
  Currency    Equiv EUR
0      EUR   25 000 000
1      EUR  465 000 000
2      EUR  123 480 766
3      EUR   36 481 250

Nombre de valeurs non vides: 4


## Étape 10 : Création de la colonne Equiv EUR

Conversion du nominal en EUR en utilisant le convertisseur de devises avec la date de création, puis formatage avec séparateur de milliers.


In [53]:
# Création colonne Equiv EUR : conversion du nominal en EUR avec formatage séparateur de milliers
def convert_to_eur_value(row):
    """Convertit un montant vers EUR en utilisant CurrencyConverter avec Creation Date"""
    amount = row['Nominal']
    currency = row['Currency']
    isin = row.get('ISIN', '')
    
    if pd.isna(amount) or pd.isna(currency):
        return np.nan
    
    amount_float = safe_float(amount)
    if pd.isna(amount_float):
        return np.nan
    
    currency_str = safe_str(currency).upper()
    if not currency_str:
        return np.nan
    
    if currency_str == 'EUR':
        return amount_float
    
    # Récupérer la Creation Date depuis le tableau final (déjà formatée)
    # Utiliser l'ISIN pour trouver la ligne correspondante dans tableau_final
    target_date = None
    if 'Creation Date' in tableau_final.columns and 'ISIN (all)' in tableau_final.columns:
        # Trouver la ligne dans tableau_final qui correspond à cet ISIN
        matching_rows = tableau_final[tableau_final['ISIN (all)'] == isin]
        if not matching_rows.empty:
            creation_date_str = matching_rows.iloc[0]['Creation Date']
            if pd.notna(creation_date_str) and str(creation_date_str).strip() != '':
                try:
                    # Parser la date au format DD-Mon-YY
                    target_date = datetime.strptime(str(creation_date_str).strip(), "%d-%b-%y").date()
                except Exception as e:
                    # En cas d'erreur de parsing, continuer sans date (utilisera le dernier taux disponible)
                    pass
    
    try:
        converted = converter.convert(amount_float, currency_str, 'EUR', target_date)
        return converted if converted is not None else np.nan
    except:
        return np.nan

tableau_final['Equiv EUR'] = ini.apply(lambda row: format_number_with_spaces(convert_to_eur_value(row)), axis=1)

# Confirmation de l'étape 10
print("=" * 80)
print("ÉTAPE 10 : Création de la colonne Equiv EUR - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Equiv EUR' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Equiv EUR':")
print(tableau_final[['Currency', 'Equiv EUR']].head(10))
print(f"\nNombre de valeurs non vides: {(tableau_final['Equiv EUR'] != '').sum()}")
print("=" * 80)


ÉTAPE 10 : Création de la colonne Equiv EUR - TERMINÉE

✓ Colonne 'Equiv EUR' créée avec 4 lignes

Aperçu de la colonne 'Equiv EUR':
  Currency    Equiv EUR
0      EUR   25 000 000
1      EUR  465 000 000
2      EUR  123 480 766
3      EUR   36 481 250

Nombre de valeurs non vides: 4


## Étape 11 : Création de la colonne Issue Price

Conversion du prix d'émission en pourcentage avec 2 décimales.


In [54]:
# Création colonne Issue Price : conversion en pourcentage avec 2 décimales
tableau_final['Issue Price'] = ini['Issue Price'].apply(format_percentage)

# Confirmation de l'étape 11
print("=" * 80)
print("ÉTAPE 11 : Création de la colonne Issue Price - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Issue Price' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Issue Price':")
print(tableau_final[['Issue Price']].head(10))
print(f"\nNombre de valeurs non vides: {(tableau_final['Issue Price'] != '').sum()}")
print("=" * 80)


ÉTAPE 11 : Création de la colonne Issue Price - TERMINÉE

✓ Colonne 'Issue Price' créée avec 4 lignes

Aperçu de la colonne 'Issue Price':
  Issue Price
0     100.00%
1     100.00%
2      75.33%
3      68.53%

Nombre de valeurs non vides: 4


## Étape 12 : Création de la colonne Collat Name

Mapping des noms de collatéraux via le dictionnaire de correspondance et ajout du suffixe "I/L" si inflation linked.


In [55]:
# Création colonne Collat Name : mapping via dictionnaire et ajout "I/L" si inflation linked
def process_collat_name(row):
    collat_name_val = row.get('Collat Name', '')
    inflation_linked = row.get('Inflation Linked? (Collat)', '')
    if pd.isna(collat_name_val):
        return ""
    collat_str = safe_str(collat_name_val)
    if '|' in collat_str:
        parts = [p.strip() for p in collat_str.split('|')]
        mapped_parts = [collat_name.get(part, part) for part in parts]
        return " | ".join(mapped_parts)
    else:
        mapped_name = collat_name.get(collat_str, collat_str)
        inflation_str = safe_str(inflation_linked).upper()
        if inflation_str == 'YES':
            mapped_name += " I/L"
        return mapped_name

tableau_final['Collat Name'] = ini.apply(process_collat_name, axis=1)

# Confirmation de l'étape 12
print("=" * 80)
print("ÉTAPE 12 : Création de la colonne Collat Name - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Collat Name' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Collat Name':")
print(tableau_final[['Collat Name']].head(10))
print(f"\nNombre de valeurs non vides: {(tableau_final['Collat Name'] != '').sum()}")
print("=" * 80)


ÉTAPE 12 : Création de la colonne Collat Name - TERMINÉE

✓ Colonne 'Collat Name' créée avec 4 lignes

Aperçu de la colonne 'Collat Name':
                 Collat Name
0                    BTP I/L
1     Republic of France I/L
2              Basket of Gov
3  The Republic of Italy I/L

Nombre de valeurs non vides: 4


## Étape 13 : Création de la colonne Collat ISIN

Gestion des valeurs séparées par "|" pour les ISIN de collatéraux.


In [56]:
# Création colonne Collat ISIN : gestion des valeurs séparées par "|"
tableau_final['Collat ISIN'] = ini['Collat ISIN'].apply(lambda x: process_pipe_separated(x))

# Confirmation de l'étape 13
print("=" * 80)
print("ÉTAPE 13 : Création de la colonne Collat ISIN - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Collat ISIN' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Collat ISIN':")
print(tableau_final[['Collat ISIN']].head(10))
print(f"\nNombre de valeurs non vides: {(tableau_final['Collat ISIN'] != '').sum()}")
print("=" * 80)


ÉTAPE 13 : Création de la colonne Collat ISIN - TERMINÉE

✓ Colonne 'Collat ISIN' créée avec 4 lignes

Aperçu de la colonne 'Collat ISIN':
                                         Collat ISIN
0                                       IT0005647273
1                                       FR0014001881
2  ES0000012E51 | ES0000012932 | ES0000012L60 | E...
3                                       IT0005547812

Nombre de valeurs non vides: 4


## Étape 14 : Création de la colonne Collat CCY

Gestion des valeurs séparées par "|" pour les devises des collatéraux.


In [57]:
# Création colonne Collat CCY : gestion des valeurs séparées par "|"
tableau_final['Collat CCY'] = ini['Collat CCY'].apply(lambda x: process_pipe_separated(x))

# Confirmation de l'étape 14
print("=" * 80)
print("ÉTAPE 14 : Création de la colonne Collat CCY - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Collat CCY' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Collat CCY':")
print(tableau_final[['Collat CCY']].head(10))
print(f"\nNombre de valeurs non vides: {(tableau_final['Collat CCY'] != '').sum()}")
print("=" * 80)


ÉTAPE 14 : Création de la colonne Collat CCY - TERMINÉE

✓ Colonne 'Collat CCY' créée avec 4 lignes

Aperçu de la colonne 'Collat CCY':
              Collat CCY
0                    EUR
1                    EUR
2  EUR | EUR | EUR | EUR
3                    EUR

Nombre de valeurs non vides: 4


## Étape 15 : Création de la colonne Levrage

Calcul du levier (collatéral en EUR / Equiv EUR) en pourcentage, avec gestion des conversions de devises et des valeurs multiples séparées par "|".


In [ ]:
# Création colonne Levrage : calcul du levier (collat en EUR / Equiv EUR) en pourcentage
def calculate_leverage_multiple(row):
    """Calcule le levier, gère tous les types de données - utilise la même méthodologie que Equiv EUR"""
    collat_amount = row.get('Collat Amount', '')
    collat_ccy = row.get('Collat CCY', '')
    nominal = row['Nominal']
    currency = row['Currency']
    isin = row.get('ISIN', '')
    
    if pd.isna(collat_amount) or pd.isna(collat_ccy) or pd.isna(nominal) or pd.isna(currency):
        return ""
    
    collat_amount_str = safe_str(collat_amount)
    collat_ccy_str = safe_str(collat_ccy)
    
    # Récupérer la Creation Date depuis le tableau final (déjà formatée)
    # Utiliser l'ISIN pour trouver la ligne correspondante dans tableau_final
    target_date = None
    if 'Creation Date' in tableau_final.columns and 'ISIN (all)' in tableau_final.columns:
        # Trouver la ligne dans tableau_final qui correspond à cet ISIN
        matching_rows = tableau_final[tableau_final['ISIN (all)'] == isin]
        if not matching_rows.empty:
            creation_date_str = matching_rows.iloc[0]['Creation Date']
            if pd.notna(creation_date_str) and str(creation_date_str).strip() != '':
                try:
                    # Parser la date au format DD-Mon-YY
                    target_date = datetime.strptime(str(creation_date_str).strip(), "%d-%b-%y").date()
                except Exception as e:
                    # En cas d'erreur de parsing, continuer sans date (utilisera le dernier taux disponible)
                    pass
    
    # Convertir le nominal en EUR (utiliser Equiv EUR si disponible, sinon calculer)
    equiv_eur = None
    if 'Equiv EUR' in tableau_final.columns:
        matching_rows = tableau_final[tableau_final['ISIN (all)'] == isin]
        if not matching_rows.empty:
            equiv_eur_str = matching_rows.iloc[0]['Equiv EUR']
            if pd.notna(equiv_eur_str) and str(equiv_eur_str).strip() != '':
                # Enlever les espaces de formatage et convertir
                equiv_eur_str_clean = str(equiv_eur_str).replace(' ', '')
                try:
                    equiv_eur = safe_float(equiv_eur_str_clean)
                except:
                    pass
    
    # Si Equiv EUR n'est pas disponible, le calculer
    if equiv_eur is None or pd.isna(equiv_eur):
        nominal_float = safe_float(nominal)
        currency_str = safe_str(currency).upper()
        if pd.isna(nominal_float) or not currency_str:
            return ""
        
        if currency_str == 'EUR':
            equiv_eur = nominal_float
        else:
            try:
                equiv_eur = converter.convert(nominal_float, currency_str, 'EUR', target_date)
                if equiv_eur is None or pd.isna(equiv_eur):
                    return ""
            except:
                return ""
    
    if equiv_eur == 0:
        return ""
    
    if '|' not in collat_amount_str and '|' not in collat_ccy_str:
        # Cas simple : une seule valeur
        collat_amount_float = safe_float(collat_amount)
        collat_ccy_str_single = safe_str(collat_ccy).upper()
        if pd.isna(collat_amount_float) or not collat_ccy_str_single:
            return ""
        
        if collat_ccy_str_single == 'EUR':
            collat_eur = collat_amount_float
        else:
            try:
                collat_eur = converter.convert(collat_amount_float, collat_ccy_str_single, 'EUR', target_date)
                if collat_eur is None or pd.isna(collat_eur):
                    return ""
            except:
                return ""
        
        leverage = (collat_eur / equiv_eur) * 100
        return f"{leverage:.2f}%"
    
    # Cas avec plusieurs valeurs séparées par |
    amounts = [a.strip() for a in collat_amount_str.split('|')]
    ccies = [c.strip() for c in collat_ccy_str.split('|')]
    leverages = []
    for i, amount in enumerate(amounts):
        if i < len(ccies):
            ccy = safe_str(ccies[i]).upper()
            if not ccy:
                continue
            try:
                amount_float = safe_float(amount)
                if pd.isna(amount_float):
                    continue
                
                if ccy == 'EUR':
                    collat_eur = amount_float
                else:
                    collat_eur = converter.convert(amount_float, ccy, 'EUR', target_date)
                    if collat_eur is None or pd.isna(collat_eur):
                        continue
                
                leverage = (collat_eur / equiv_eur) * 100
                leverages.append(f"{leverage:.2f}%")
            except:
                pass
    return " | ".join(leverages) if leverages else ""

tableau_final['Levrage'] = ini.apply(calculate_leverage_multiple, axis=1)

# Confirmation de l'étape 15
print("=" * 80)
print("ÉTAPE 15 : Création de la colonne Levrage - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Levrage' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Levrage':")
print(tableau_final[['Equiv EUR', 'Levrage']].head(10))
print(f"\nNombre de valeurs non vides: {(tableau_final['Levrage'] != '').sum()}")
print("=" * 80)


ÉTAPE 15 : Création de la colonne Levrage - TERMINÉE

✓ Colonne 'Levrage' créée avec 4 lignes

Aperçu de la colonne 'Levrage':
     Equiv EUR Levrage
0   25 000 000  98.00%
1  465 000 000  13.96%
2  123 480 766        
3   36 481 250  86.62%

Nombre de valeurs non vides: 3


## Étape 16 : Création de la colonne Coupon

Formatage des coupons selon le type : Fixed, Floating, Variable-linked ou Zero Coupon.


In [59]:
# Création colonne Coupon : formatage selon le type (Fixed, Floating, Variable-linked, Zero Coupon)
def process_coupon(row):
    """Traite les coupons, gère tous les types de données"""
    interest_basis = safe_str(row.get('Interest Basis', '')).strip()
    if interest_basis == 'Fixed':
        fixed_note = row.get('Fixed Note', '')
        if pd.notna(fixed_note):
            return f"Y1 - End: {safe_str(fixed_note)}% p.a."
        return ""
    elif interest_basis == 'Fixed, Variable-linked':
        fixed_note = row.get('Fixed Note', '')
        variable_note = row.get('Variable-linked Note', '')
        return format_variable_linked_coupon(fixed_note, variable_note)
    elif interest_basis == 'Floating':
        floating_note = row.get('Floating Note', '')
        return format_floating_coupon(floating_note)
    elif interest_basis == 'Zero Coupon':
        issue_price = row.get('Issue Price', np.nan)
        maturity_date = row.get('Maturity Date', np.nan)
        issue_date = row.get('Issue Date', np.nan)
        if pd.notna(issue_price) and pd.notna(maturity_date) and pd.notna(issue_date):
            try:
                maturity = safe_date(maturity_date)
                issue = safe_date(issue_date)
                if maturity is None or issue is None:
                    return ""
                years_diff = (maturity - issue).days / 365.25
                issue_price_float = safe_float(issue_price)
                if not pd.isna(issue_price_float) and issue_price_float > 0 and years_diff > 0:
                    irr = -1 + (1 / issue_price_float) ** (1 / years_diff)
                    irr_percent = irr * 100
                    return f"ZC - {irr_percent:.2f}% IRR"
            except Exception as e:
                pass
        return ""
    return ""

tableau_final['Coupon'] = ini.apply(process_coupon, axis=1)

# Confirmation de l'étape 16
print("=" * 80)
print("ÉTAPE 16 : Création de la colonne Coupon - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Coupon' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Coupon':")
print(tableau_final[['Coupon']].head(10))
print(f"\nNombre de valeurs non vides: {(tableau_final['Coupon'] != '').sum()}")
print("=" * 80)


ÉTAPE 16 : Création de la colonne Coupon - TERMINÉE

✓ Colonne 'Coupon' créée avec 4 lignes

Aperçu de la colonne 'Coupon':
                                              Coupon
0  Y1 - End: Min(6.00% ; Max(EUR6M + 3.14% ; 0.00%))
1                               Y1 - End: 4.88% p.a.
2                                     ZC - 1.69% IRR
3                                     ZC - 2.26% IRR

Nombre de valeurs non vides: 4


## Étape 17 : Création de la colonne Final Redemption

Conversion du remboursement final en pourcentage si nécessaire (gestion des valeurs décimales et pourcentages).


In [60]:
# Création colonne Final Redemption : conversion en pourcentage si nécessaire
def process_final_redemption(value):
    """Traite Final Redemption, gère tous les types de données"""
    if pd.isna(value):
        return ""
    value_str = safe_str(value)
    if '%' in value_str:
        return value_str
    try:
        num_value = safe_float(value)
        if not pd.isna(num_value):
            if 0 <= num_value <= 1:
                return f"{num_value * 100:.2f}%"
            elif 1 < num_value <= 100:
                return f"{num_value:.2f}%"
    except:
        pass
    return ""

tableau_final['Final Redemption'] = ini['Final Redemption'].apply(process_final_redemption)

# Confirmation de l'étape 17
print("=" * 80)
print("ÉTAPE 17 : Création de la colonne Final Redemption - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Final Redemption' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Final Redemption':")
print(tableau_final[['Final Redemption']].head(10))
print(f"\nNombre de valeurs non vides: {(tableau_final['Final Redemption'] != '').sum()}")
print("=" * 80)


ÉTAPE 17 : Création de la colonne Final Redemption - TERMINÉE

✓ Colonne 'Final Redemption' créée avec 4 lignes

Aperçu de la colonne 'Final Redemption':
  Final Redemption
0             100%
1             100%
2                 
3                 

Nombre de valeurs non vides: 2


## Étape 18 : Création de la colonne Other comments

Ajout des commentaires sur les options d'appel émetteur (Issuer Call) et les options de changement (Issuer Switch Option) si présents.


In [ ]:
# Création colonne Other comments : Issuer Call et Issuer Switch Option si présents
def process_other_comments(row):
    """Traite Other comments, gère tous les types de données"""
    comments = []
    issuer_call_date = row.get('Issuer Call Redemption Date', '')
    issuer_call_amount = row.get('Issuer Call Redemption Amount', '')
    
    issuer_call_date_str = safe_str(issuer_call_date).upper()
    if pd.notna(issuer_call_date) and issuer_call_date_str != 'N/A':
        # Formater la date en supprimant "00:00:00" si présent
        date_str = safe_str(issuer_call_date)
        # Supprimer "00:00:00" et les espaces autour
        if '00:00:00' in date_str:
            date_str = date_str.replace('00:00:00', '').strip()
        # Si c'est un datetime, formater en date seulement
        try:
            if isinstance(issuer_call_date, (datetime, pd.Timestamp)):
                date_str = issuer_call_date.strftime('%Y-%m-%d')
            elif isinstance(issuer_call_date, date):
                date_str = issuer_call_date.strftime('%Y-%m-%d')
        except:
            pass
        issuer_call_amount_str = safe_str(issuer_call_amount).upper()
        if pd.notna(issuer_call_amount) and issuer_call_amount_str != 'N/A':
            try:
                # Convertir le montant en pourcentage entier
                amount_value = safe_float(issuer_call_amount)
                if not pd.isna(amount_value):
                    if 0 <= amount_value <= 1:
                        amount_str = f"{safe_int(amount_value * 100)}%"
                    elif 1 < amount_value <= 100:
                        amount_str = f"{safe_int(amount_value)}%"
                    else:
                        amount_str = safe_str(issuer_call_amount)
                else:
                    amount_str = safe_str(issuer_call_amount)
            except:
                amount_str = safe_str(issuer_call_amount)
            comments.append(f"Issuer Call {date_str} @{amount_str}")
        else:
            comments.append(f"Issuer Call {date_str}")
    type_of_coupon = safe_str(row.get('Type of coupon', ''))
    if pd.notna(row.get('Type of coupon', '')) and ',' in type_of_coupon:
        comments.append("Issuer Switch Option YYYY-MM-DD")
    return " ".join(comments) if comments else ""

tableau_final['Other comments'] = ini.apply(process_other_comments, axis=1)

# Confirmation de l'étape 18
print("=" * 80)
print("ÉTAPE 18 : Création de la colonne Other comments - TERMINÉE")
print("=" * 80)
print(f"\n✓ Colonne 'Other comments' créée avec {len(tableau_final)} lignes")
print(f"\nAperçu de la colonne 'Other comments':")
print(tableau_final[['Other comments']].head(10))
print(f"\nNombre de valeurs non vides: {(tableau_final['Other comments'] != '').sum()}")
print("=" * 80)


ÉTAPE 18 : Création de la colonne Other comments - TERMINÉE

✓ Colonne 'Other comments' créée avec 4 lignes

Aperçu de la colonne 'Other comments':
                 Other comments
0                              
1  Issuer Call 2042-09-15 @100%
2                              
3                              

Nombre de valeurs non vides: 1


## Étape 19 : Vérification, affichage et sauvegarde du tableau final

Vérification que le tableau final contient bien 15 colonnes, affichage du résultat et sauvegarde au format Excel dans le dossier "data/Tableau final".


In [ ]:
# Vérification du nombre de colonnes (doit être 15) et affichage du tableau final
if len(tableau_final.columns) != 15:
    import warnings
    warnings.warn(f"ATTENTION: Le tableau final contient {len(tableau_final.columns)} colonnes au lieu de 15 attendues. Colonnes: {list(tableau_final.columns)}")

# Confirmation de l'étape 19
print("=" * 80)
print("ÉTAPE 19 : Vérification et affichage du tableau final - TERMINÉE")
print("=" * 80)
print(f"\n✓ Nombre de colonnes: {len(tableau_final.columns)}")
print(f"✓ Nombre de lignes: {len(tableau_final)}")
print(f"\nColonnes du tableau final:")
for i, col in enumerate(tableau_final.columns, 1):
    print(f"  {i}. {col}")
print(f"\nAperçu complet du tableau final:")
print(tableau_final.head(10))

# Création du dossier "Tableau final" dans data/ et sauvegarde au format Excel
try:
    # Créer le chemin du dossier
    output_dir = Path('data/Tableau final')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Générer un nom de fichier avec la date et l'heure
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = output_dir / f"tableau_final_{timestamp}.xlsx"
    
    # Sauvegarder le tableau final au format Excel
    tableau_final.to_excel(output_file, index=False, engine='openpyxl')
    
    print(f"\n✓ Tableau final sauvegardé avec succès")
    print(f"  Chemin: {output_file}")
    print(f"  Taille: {len(tableau_final)} lignes × {len(tableau_final.columns)} colonnes")
except Exception as e:
    print(f"\n❌ Erreur lors de la sauvegarde: {e}")
    import traceback
    traceback.print_exc()

print("=" * 80)


                                  Dealer    ISIN (all) N° Issuance  \
0                                   BNPP  XS2030639145        2654   
1                          HSBC Bank plc  XS2135238659        2543   
2                         J.P. Morgan SE  XS2041123880        2020   
3  Nomura Financial Products Europe GmbH  XS3205809513        4343   

  Creation Date   Maturity Currency    Equiv EUR Issue Price  \
0     17-Dec-24  15-May-56      EUR   25 000 000     100.00%   
1     18-Nov-23  25-Jul-53      EUR  465 000 000     100.00%   
2     25-May-35  15-Sep-42      EUR  123 480 766      75.33%   
3     18-Dec-24  15-Sep-42      EUR   36 481 250      68.53%   

                 Collat Name  \
0                    BTP I/L   
1     Republic of France I/L   
2              Basket of Gov   
3  The Republic of Italy I/L   

                                         Collat ISIN             Collat CCY  \
0                                       IT0005647273                    EUR   
1        

## Étape 20 : Sauvegarde du tableau final

Enregistrement du tableau final au format Excel dans le dossier "data/Tableau final" avec un nom de fichier incluant la date et l'heure.


In [ ]:
# Sauvegarde du tableau final au format Excel dans le dossier "data/Tableau final"
print("=" * 80)
print("ÉTAPE 20 : Sauvegarde du tableau final - EN COURS")
print("=" * 80)

try:
    # Créer le chemin du dossier
    output_dir = Path('data/Tableau final')
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"\n✓ Dossier créé/vérifié: {output_dir}")
    
    # Générer un nom de fichier avec la date et l'heure
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = output_dir / f"tableau_final_{timestamp}.xlsx"
    
    # Sauvegarder le tableau final au format Excel
    tableau_final.to_excel(output_file, index=False, engine='openpyxl')
    
    print(f"\n✓ Tableau final sauvegardé avec succès")
    print(f"  Chemin: {output_file}")
    print(f"  Taille: {len(tableau_final)} lignes × {len(tableau_final.columns)} colonnes")
    print(f"  Colonnes: {list(tableau_final.columns)}")
    
except Exception as e:
    print(f"\n❌ Erreur lors de la sauvegarde: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 80)
print("ÉTAPE 20 : Sauvegarde du tableau final - TERMINÉE")
print("=" * 80)


## Étape 21 : Tests des principales fonctions de CurrencyConverter

Tests minimaux des principales fonctions publiques de la classe CurrencyConverter.


In [63]:
# Tests des principales fonctions de CurrencyConverter
print("=" * 80)
print("TESTS DES PRINCIPALES FONCTIONS DE CurrencyConverter")
print("=" * 80)

# Test 1: import_rates - Import d'un taux pour une date
print("\n[TEST] import_rates - Import d'un taux EUR_USD pour une date")
try:
    test_date = date(2024, 1, 15)
    df_rates = converter.import_rates("EUR_USD", target_date=test_date)
    print(f"✓ import_rates('EUR_USD', target_date={test_date})")
    print(f"  Résultat: DataFrame avec {len(df_rates)} lignes")
    if not df_rates.empty:
        print(f"  Colonnes: {list(df_rates.columns)}")
        print(df_rates.head())
except Exception as e:
    print(f"✗ Erreur: {e}")

# Test 2: import_rates - Import de plusieurs paires pour une plage de dates
print("\n[TEST] import_rates - Import de plusieurs paires pour une plage de dates")
try:
    start_date = date(2024, 1, 1)
    end_date = date(2024, 1, 31)
    df_rates = converter.import_rates(["EUR_USD", "EUR_GBP"], start_date=start_date, end_date=end_date)
    print(f"✓ import_rates(['EUR_USD', 'EUR_GBP'], start_date={start_date}, end_date={end_date})")
    print(f"  Résultat: DataFrame avec {len(df_rates)} lignes")
    if not df_rates.empty:
        print(f"  Colonnes: {list(df_rates.columns)}")
        print(df_rates.head())
except Exception as e:
    print(f"✗ Erreur: {e}")

# Test 3: convert - Conversion simple EUR vers USD
print("\n[TEST] convert - Conversion EUR vers USD")
try:
    result = converter.convert(100, "EUR", "USD", date(2024, 1, 15))
    print(f"✓ convert(100, 'EUR', 'USD', date(2024, 1, 15))")
    print(f"  Résultat: {result}")
except Exception as e:
    print(f"✗ Erreur: {e}")

# Test 4: convert - Conversion USD vers EUR
print("\n[TEST] convert - Conversion USD vers EUR")
try:
    result = converter.convert(100, "USD", "EUR", date(2024, 1, 15))
    print(f"✓ convert(100, 'USD', 'EUR', date(2024, 1, 15))")
    print(f"  Résultat: {result}")
except Exception as e:
    print(f"✗ Erreur: {e}")

# Test 5: convert - Conversion entre deux devises non-EUR (via EUR)
print("\n[TEST] convert - Conversion USD vers GBP (via EUR)")
try:
    result = converter.convert(100, "USD", "GBP", date(2024, 1, 15))
    print(f"✓ convert(100, 'USD', 'GBP', date(2024, 1, 15))")
    print(f"  Résultat: {result}")
except Exception as e:
    print(f"✗ Erreur: {e}")

# Test 6: convert - Conversion sans date (utilise le dernier taux disponible)
print("\n[TEST] convert - Conversion sans date (dernier taux disponible)")
try:
    result = converter.convert(100, "EUR", "USD", None)
    print(f"✓ convert(100, 'EUR', 'USD', None)")
    print(f"  Résultat: {result}")
except Exception as e:
    print(f"✗ Erreur: {e}")

# Test 7: convert - Conversion même devise
print("\n[TEST] convert - Conversion même devise (EUR vers EUR)")
try:
    result = converter.convert(100, "EUR", "EUR", date(2024, 1, 15))
    print(f"✓ convert(100, 'EUR', 'EUR', date(2024, 1, 15))")
    print(f"  Résultat: {result} (doit être 100)")
except Exception as e:
    print(f"✗ Erreur: {e}")

print("\n" + "=" * 80)
print("FIN DES TESTS")
print("=" * 80)


TESTS DES PRINCIPALES FONCTIONS DE CurrencyConverter

[TEST] import_rates - Import d'un taux EUR_USD pour une date
⚠ Aucune donnée disponible dans le backup
✓ import_rates('EUR_USD', target_date=2024-01-15)
  Résultat: DataFrame avec 0 lignes

[TEST] import_rates - Import de plusieurs paires pour une plage de dates
⚠ Aucune donnée disponible dans le backup
✓ import_rates(['EUR_USD', 'EUR_GBP'], start_date=2024-01-01, end_date=2024-01-31)
  Résultat: DataFrame avec 0 lignes

[TEST] convert - Conversion EUR vers USD
⚠ Aucune donnée disponible dans le backup
⚠ Aucun taux de change trouvé pour EUR -> USD à la date 2024-01-15
✓ convert(100, 'EUR', 'USD', date(2024, 1, 15))
  Résultat: None

[TEST] convert - Conversion USD vers EUR
⚠ Aucune donnée disponible dans le backup
⚠ Aucun taux de change trouvé pour USD -> EUR à la date 2024-01-15
✓ convert(100, 'USD', 'EUR', date(2024, 1, 15))
  Résultat: None

[TEST] convert - Conversion USD vers GBP (via EUR)
⚠ Aucune donnée disponible dans le bac